# 모델 저장 및 학습 중단

In [2]:
# python에서 yaml을 사용하기 위한 모듈
!pip install pyyaml h5py

## 데이터 로드

In [35]:
import tensorflow as tf
from tensorflow import keras

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# 모델 실행 속도를 빠르게하기 위해 샘플 데이터 1000개만 사용
train_images=train_images[:1000].reshape(-1,28*28)/255.0
test_images=test_images[:1000].reshape(-1,28*28)/255.0

train_labels=train_labels[:1000]
test_labels=test_labels[:1000]

## 모델 정의

In [9]:
def create_model():
    model=tf.keras.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

model=create_model()
model.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-14 00:45:11.044719: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 00:45:11.044924: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 훈련하는 동안 체크포인트 저장하기
- tf.kerass.callbacks.ModelCheckpoint 콜백을 사용하면 훈련 도중 또는 훈련 종료 시 모델을 저장할 수 있다.

In [52]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 베스트 모델 저장 폴더
checkpoint_path="model/cp.ckpt"
checkpoint_dir=os.path.dirname(checkpoint_path)

# 베스트 모델 저장 : 학습 중 나타난 가장 좋은 모델 저장
MCP=ModelCheckpoint(
    filepath=checkpoint_path,  # 파일이 저장될 경로
    monitor='val_loss',  # train set의 loss가 가장 작을 때 'loss'를 입력 / validation set의 loss가 가장 작을 때 저장하고 싶으면 'val_loss'를 입력
    save_best_only=True,  # 모델이 개선되었을 때만 저장
    verbovse=1  # 진행 내용 출력
)

# 학습 중단 : 모델이 더이상 개선되지 않으면 학습 중단
ES=EarlyStopping(
    monitor='val_accuracy',  # 무엇을 기준으로 개선여부 판단할지 설정 분류는'val_accuracy' / 회귀는 'val_loss'
    patience=5   # 인내 값 : 연속으로 몇번의 학습동안 모델이 개선되지 않아도 중단하지 않음
)

model.fit(train_images,train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[MCP,ES])


Epoch 1/10
32/32 [==============================] - 0s 10ms/step - loss: 0.0017 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4614 - val_sparse_categorical_accuracy: 0.8750
INFO:tensorflow:Assets written to: model/cp.ckpt/assets
Epoch 2/10
32/32 [==============================] - 0s 9ms/step - loss: 0.0017 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4717 - val_sparse_categorical_accuracy: 0.8700
Epoch 3/10
32/32 [==============================] - 0s 9ms/step - loss: 0.0015 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4716 - val_sparse_categorical_accuracy: 0.8740
Epoch 4/10
32/32 [==============================] - 0s 9ms/step - loss: 0.0015 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4719 - val_sparse_categorical_accuracy: 0.8720
Epoch 5/10
32/32 [==============================] - 0s 9ms/step - loss: 0.0015 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4741 - val_sparse_categorical_accuracy: 0.8740
Epoch 6/10
32/32 [==============================] - 0

## 모델 평가

In [47]:
# 비교모델 생성
model2 = create_model()
loss, acc = model2.evaluate(test_images, test_labels, verbose=2)
print("Untrained, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3531 - sparse_categorical_accuracy: 0.0840
Untrained, accuracy:  8.40%


2023-02-14 01:29:23.290053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [53]:
# 체크포인트에서 가중치를 로드하고 다시 평가
model2.load_weights(checkpoint_path)
loss, acc = model2.evaluate(test_images, test_labels, verbose=2)
print("Restored, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4614 - sparse_categorical_accuracy: 0.8750
Restored, accuracy: 87.50%


2023-02-14 01:36:22.960104: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open model/cp.ckpt: Failed precondition: model/cp.ckpt; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?
